In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "NF-CICIDS2018-v3.parquet"
data = pd.read_parquet(file_name)

In [3]:
##ADDED
file_name_test = "NF-CSE-CIC-IDS2018-v2.parquet"
test_data = pd.read_parquet(file_name_test)
##

In [4]:
data.Label.value_counts()

Label
0    17514626
1     2600903
Name: count, dtype: int64

In [5]:
## ADDED
test_data.Label.value_counts()
##

Label
0    16635567
1     2258141
Name: count, dtype: int64

In [6]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [7]:
##ADDED
test_data.rename(columns=lambda x: x.strip(), inplace=True)
test_data['IPV4_SRC_ADDR'] = test_data["IPV4_SRC_ADDR"].apply(str)
test_data['L4_SRC_PORT'] = test_data["L4_SRC_PORT"].apply(str)
test_data['IPV4_DST_ADDR'] = test_data["IPV4_DST_ADDR"].apply(str)
test_data['L4_DST_PORT'] = test_data["L4_DST_PORT"].apply(str)
##

In [8]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [9]:
# ADDED
test_data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)
##

In [10]:
data.Attack.unique()

array(['Benign', 'FTP-BruteForce', 'SSH-Bruteforce',
       'DoS_attacks-GoldenEye', 'DoS_attacks-Slowloris',
       'DoS_attacks-SlowHTTPTest', 'DoS_attacks-Hulk',
       'DDoS_attacks-LOIC-HTTP', 'DDOS_attack-LOIC-UDP',
       'DDOS_attack-HOIC', 'Brute_Force_-Web', 'Brute_Force_-XSS',
       'SQL_Injection', 'Infilteration', 'Bot'], dtype=object)

In [11]:
##ADDED
test_data.Attack.unique()
##

array(['SSH-Bruteforce', 'Benign', 'DDoS attacks-LOIC-HTTP',
       'DDOS attack-HOIC', 'DoS attacks-Slowloris', 'DoS attacks-Hulk',
       'FTP-BruteForce', 'Infilteration', 'Bot', 'DoS attacks-GoldenEye',
       'Brute Force -Web', 'DoS attacks-SlowHTTPTest', 'SQL Injection',
       'DDOS attack-LOIC-UDP', 'Brute Force -XSS'], dtype=object)

In [12]:
## ADDED
test_data['Attack'] = test_data['Attack'].str.replace(' ', '_')
##

In [13]:
data = data.groupby(by='Attack').sample(frac=0.02, random_state=13)

In [14]:
## ADDED
test_data = test_data.groupby(by='Attack').sample(frac=0.02, random_state=13)
##

In [15]:
data.groupby(by="Attack").count()

,FLOW_START_MILLISECONDS,FLOW_END_MILLISECONDS,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,FTP_COMMAND_RET_CODE,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,350293,350293,350293,350293,350293,350293,350293,350293,350293,350293,...,350293,350293,350293,350293,350293,350293,350293,350293,350293,350293
Bot,4154,4154,4154,4154,4154,4154,4154,4154,4154,4154,...,4154,4154,4154,4154,4154,4154,4154,4154,4154,4154
Brute_Force_-Web,32,32,32,32,32,32,32,32,32,32,...,32,32,32,32,32,32,32,32,32,32
Brute_Force_-XSS,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
DDOS_attack-HOIC,20646,20646,20646,20646,20646,20646,20646,20646,20646,20646,...,20646,20646,20646,20646,20646,20646,20646,20646,20646,20646
DDOS_attack-LOIC-UDP,69,69,69,69,69,69,69,69,69,69,...,69,69,69,69,69,69,69,69,69,69
DDoS_attacks-LOIC-HTTP,5772,5772,5772,5772,5772,5772,5772,5772,5772,5772,...,5772,5772,5772,5772,5772,5772,5772,5772,5772,5772
DoS_attacks-GoldenEye,1226,1226,1226,1226,1226,1226,1226,1226,1226,1226,...,1226,1226,1226,1226,1226,1226,1226,1226,1226,1226
DoS_attacks-Hulk,2002,2002,2002,2002,2002,2002,2002,2002,2002,2002,...,2002,2002,2002,2002,2002,2002,2002,2002,2002,2002


In [16]:
##ADDED
test_data.groupby(by="Attack").count()
##

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,332711,332711,332711,332711,332711,332711,332711,332711,332711,332711,...,332711,332711,332711,332711,332711,332711,332711,332711,332711,332711
Bot,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862,...,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862
Brute_Force_-Web,43,43,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
Brute_Force_-XSS,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
DDOS_attack-HOIC,21617,21617,21617,21617,21617,21617,21617,21617,21617,21617,...,21617,21617,21617,21617,21617,21617,21617,21617,21617,21617
DDOS_attack-LOIC-UDP,42,42,42,42,42,42,42,42,42,42,...,42,42,42,42,42,42,42,42,42,42
DDoS_attacks-LOIC-HTTP,6146,6146,6146,6146,6146,6146,6146,6146,6146,6146,...,6146,6146,6146,6146,6146,6146,6146,6146,6146,6146
DoS_attacks-GoldenEye,554,554,554,554,554,554,554,554,554,554,...,554,554,554,554,554,554,554,554,554,554
DoS_attacks-Hulk,8653,8653,8653,8653,8653,8653,8653,8653,8653,8653,...,8653,8653,8653,8653,8653,8653,8653,8653,8653,8653


In [ ]:
# X = data.drop(columns=["Attack", "Label"])
X = data.drop(columns=["Attack", "Label", "FLOW_START_MILLISECONDS", "FLOW_END_MILLISECONDS",
                       "SRC_TO_DST_IAT_MIN", "SRC_TO_DST_IAT_MAX", "SRC_TO_DST_IAT_AVG",
                       "SRC_TO_DST_IAT_STDDEV", "DST_TO_SRC_IAT_MIN", "DST_TO_SRC_IAT_MAX",
                       "DST_TO_SRC_IAT_AVG", "DST_TO_SRC_IAT_STDDEV"])
y = data[["Attack", "Label"]]

# X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.3, random_state=13, stratify=y)
# ADDED
X_train, y_train = X, y
X_test = test_data.drop(columns=["Attack", "Label"])
y_test = test_data[["Attack", "Label"]]

X_train.shape, X_test.shape, y_train.shape, y_test.shape
##

((281617, 41), (377874, 41), (281617, 2), (377874, 2))

In [18]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [19]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [20]:
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [21]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h
18744204,172.31.69.14,18.219.211.138,4.740102e-07,1.323114e-06,0.001264,0.000012,0.000801,0.000012,1.045428e-06,1.016509e-06,...,0.0,0.019249,0.063169,3.165538e-07,3.165538e-07,4.626324e-07,4.626324e-07,0.000000,3.038169e-07,"[4.7401022514464873e-07, 1.3231141133088036e-0..."
16092777,172.31.64.33,169.254.169.254,8.691857e-08,2.426175e-07,0.000155,0.000002,0.000196,0.000002,1.177649e-07,1.037055e-07,...,0.0,0.003530,0.007722,5.804601e-08,5.804601e-08,8.483223e-08,8.483223e-08,0.000000,5.571047e-08,"[8.691857374655049e-08, 2.426175334037123e-07,..."
8585764,18.219.9.1,172.31.69.28,3.946205e-07,1.101512e-06,0.001047,0.000010,0.002244,0.000010,8.703343e-07,8.462588e-07,...,0.0,0.128201,0.052589,2.635357e-07,2.635357e-07,3.851483e-07,3.851483e-07,0.000000,2.529321e-07,"[3.946205413164243e-07, 1.1015121191910754e-06..."
9151507,18.218.229.235,172.31.69.28,2.759324e-07,7.702155e-07,0.000756,0.000007,0.001569,0.000007,6.085679e-07,5.917335e-07,...,0.0,0.089643,0.036772,1.842733e-07,1.842733e-07,2.693090e-07,2.693090e-07,0.000000,1.768589e-07,"[2.759323680481234e-07, 7.702154745117166e-07,..."
14212479,172.31.65.38,172.31.0.2,8.558578e-07,8.375243e-07,0.003386,0.000059,0.010988,0.000059,8.503230e-07,8.503230e-07,...,0.0,0.000000,0.000000,8.001776e-06,8.001776e-06,6.389747e-06,8.525785e-07,0.003564,7.679815e-06,"[8.558577987023315e-07, 8.375242601934382e-07,..."


In [22]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [23]:
# Training graph
train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack"], create_using=nx.MultiGraph())
train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [24]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [25]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [26]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [27]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [28]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [29]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

# dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [30]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [31]:
# Convert to GPU
# train_g = train_g.to('cuda')

In [32]:
# cnt_wait = 0
# best = 1e9
# best_t = 0
# dur = []
# node_features = train_g.ndata['h'] 
# edge_features = train_g.edata['h']
# torch.cuda.empty_cache()

# for epoch in range(epochs):
#     dgi.train()
#     if epoch >= 3:
#         t0 = time.time()

#     dgi_optimizer.zero_grad()
#     loss = dgi(train_g, node_features, edge_features)
#     loss.backward()
#     dgi_optimizer.step()

#     if loss < best:
#         best = loss
#         best_t = epoch
#         cnt_wait = 0
#         torch.save(dgi.state_dict(), 'best_dgi_CSE_v3.pkl')
#     else:
#         cnt_wait += 1

#   # if cnt_wait == patience:
#   #     print('Early stopping!')
#   #     break

#     if epoch >= 3:
#         dur.append(time.time() - t0)

#     if epoch % 50 == 0:

#         print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
#             "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
#               loss.item(),
#               train_g.num_edges() / np.mean(dur) / 1000))

In [33]:
dgi.load_state_dict(torch.load('best_dgi_CSE_v3.pkl'))

<All keys matched successfully>

In [34]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [35]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [36]:
# Convert to GPU
# test_g = test_g.to('cuda')

In [37]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [38]:
df_train = pd.DataFrame(training_emb, )
df_train["Attack"] = lab_enc.inverse_transform(
        train_g.edata['Attack'].detach().cpu().numpy())
df_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb, )
df_test["Attack"] = lab_enc.inverse_transform(
        test_g.edata['Attack'].detach().cpu().numpy())
df_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

In [39]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,Attack,Label
0,-0.043389,-0.035837,0.029989,0.023889,0.041586,0.037156,-0.008817,-0.046635,0.021885,-0.108355,...,-0.050719,-0.081328,0.066993,0.066250,0.060268,0.002366,-0.074248,0.036667,Bot,1
1,-0.043389,-0.035837,0.029989,0.023889,0.041586,0.037156,-0.008817,-0.046635,0.021885,-0.108355,...,-0.050719,-0.081328,0.066993,0.066250,0.060268,0.002366,-0.074248,0.036667,Bot,1
2,-0.043389,-0.035837,0.029989,0.023889,0.041586,0.037156,-0.008817,-0.046635,0.021885,-0.108355,...,-0.050719,-0.081328,0.066993,0.066250,0.060268,0.002366,-0.074248,0.036667,Bot,1
3,-0.043389,-0.035837,0.029989,0.023889,0.041586,0.037156,-0.008817,-0.046635,0.021885,-0.108355,...,-0.050719,-0.081328,0.066993,0.066250,0.060268,0.002366,-0.074248,0.036667,Bot,1
4,-0.043389,-0.035837,0.029989,0.023889,0.041586,0.037156,-0.008817,-0.046635,0.021885,-0.108355,...,-0.050719,-0.081328,0.066993,0.066250,0.060268,0.002366,-0.074248,0.036667,Bot,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561750,-0.043716,0.032430,-0.031950,0.025078,0.050373,0.060781,-0.000040,-0.091699,-0.023817,-0.100970,...,-0.146869,-0.075696,0.105807,0.121036,0.005141,-0.063235,-0.071510,0.064431,Benign,0
561751,-0.053499,0.026729,0.011206,0.071818,0.070160,0.040723,-0.025985,-0.065670,-0.002901,-0.074505,...,-0.085718,-0.164499,0.125317,0.124130,0.073431,-0.044648,-0.066476,0.031605,Benign,0
561752,-0.042288,0.010935,0.022614,0.059888,0.068790,0.033697,-0.034046,-0.058445,-0.001781,-0.093167,...,-0.061659,-0.170744,0.093812,0.126206,0.090327,-0.000545,-0.051569,0.029330,Benign,0
561753,0.010087,-0.019234,-0.009873,0.002050,0.069912,0.009080,0.004433,-0.103044,-0.045608,-0.077446,...,-0.140584,-0.058758,0.090740,0.076589,0.044143,-0.075119,-0.079248,-0.003991,Benign,0


# Embeddings CBLOF  Embeddings

In [40]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [41]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [42]:
test_samples

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,0.002157,-0.053237,0.026422,-0.014746,0.043351,0.001298,-0.033976,-0.061787,0.039582,-0.097414,...,0.017344,0.036406,-0.112174,-0.083311,0.082590,0.108380,0.035611,-0.020556,-0.063788,-0.011103
1,0.002157,-0.053237,0.026422,-0.014746,0.043351,0.001298,-0.033976,-0.061787,0.039582,-0.097414,...,0.017344,0.036406,-0.112174,-0.083311,0.082590,0.108380,0.035611,-0.020556,-0.063788,-0.011103
2,0.002157,-0.053237,0.026422,-0.014746,0.043351,0.001298,-0.033976,-0.061787,0.039582,-0.097414,...,0.017344,0.036406,-0.112174,-0.083311,0.082590,0.108380,0.035611,-0.020556,-0.063788,-0.011103
3,0.002157,-0.053237,0.026422,-0.014746,0.043351,0.001298,-0.033976,-0.061787,0.039582,-0.097414,...,0.017344,0.036406,-0.112174,-0.083311,0.082590,0.108380,0.035611,-0.020556,-0.063788,-0.011103
4,0.002157,-0.053237,0.026422,-0.014746,0.043351,0.001298,-0.033976,-0.061787,0.039582,-0.097414,...,0.017344,0.036406,-0.112174,-0.083311,0.082590,0.108380,0.035611,-0.020556,-0.063788,-0.011103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754753,0.059153,-0.046284,-0.020000,-0.031556,0.072498,0.003264,-0.019767,-0.120256,0.034262,-0.053864,...,0.052913,0.047198,-0.148034,-0.069749,0.084468,0.172803,-0.016983,-0.103731,-0.105667,-0.050710
754754,0.059153,-0.046284,-0.020000,-0.031556,0.072498,0.003264,-0.019767,-0.120256,0.034262,-0.053864,...,0.052913,0.047198,-0.148034,-0.069749,0.084468,0.172803,-0.016983,-0.103731,-0.105667,-0.050710
754755,0.059153,-0.046284,-0.020000,-0.031556,0.072498,0.003264,-0.019767,-0.120256,0.034262,-0.053864,...,0.052913,0.047198,-0.148034,-0.069749,0.084468,0.172803,-0.016983,-0.103731,-0.105667,-0.050710
754756,0.059153,-0.046284,-0.020000,-0.031556,0.072498,0.003264,-0.019767,-0.120256,0.034262,-0.053864,...,0.052913,0.047198,-0.148034,-0.069749,0.084468,0.172803,-0.016983,-0.103731,-0.105667,-0.050710


In [43]:
df_raw_train = pd.concat([X_train.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_train], axis=1)
df_raw_test = pd.concat([X_test.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_test], axis=1)

In [44]:
df_raw_train

,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Attack,Label
18744204,4.740102e-07,1.323114e-06,0.001264,1.174887e-05,0.000801,1.174887e-05,1.045428e-06,1.016509e-06,1.189494e-06,2.584752e-05,...,0.019249,0.063169,3.165538e-07,3.165538e-07,4.626324e-07,4.626324e-07,0.000000,3.038169e-07,Bot,1
16092777,8.691857e-08,2.426175e-07,0.000155,2.154374e-06,0.000196,2.154374e-06,1.177649e-07,1.037055e-07,2.181158e-07,4.308748e-07,...,0.003530,0.007722,5.804601e-08,5.804601e-08,8.483223e-08,8.483223e-08,0.000000,5.571047e-08,Benign,0
8585764,3.946205e-07,1.101512e-06,0.001047,9.781112e-06,0.002244,9.781112e-06,8.703343e-07,8.462588e-07,9.902714e-07,3.716823e-05,...,0.128201,0.052589,2.635357e-07,2.635357e-07,3.851483e-07,3.851483e-07,0.000000,2.529321e-07,DDOS_attack-HOIC,1
9151507,2.759324e-07,7.702155e-07,0.000756,6.839293e-06,0.001569,6.839293e-06,6.085679e-07,5.917335e-07,6.924321e-07,1.778216e-05,...,0.089643,0.036772,1.842733e-07,1.842733e-07,2.693090e-07,2.693090e-07,0.000000,1.768589e-07,DDOS_attack-HOIC,1
14212479,8.558578e-07,8.375243e-07,0.003386,5.939708e-05,0.010988,5.939708e-05,8.503230e-07,8.503230e-07,8.198402e-07,5.405134e-03,...,0.000000,0.000000,8.001776e-06,8.001776e-06,6.389747e-06,8.525785e-07,0.003564,7.679815e-06,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7181547,5.916211e-07,5.789478e-07,0.002792,4.105888e-05,0.006282,4.105888e-05,5.877951e-07,5.877951e-07,5.667235e-07,2.217180e-03,...,0.000000,0.000000,5.531315e-06,5.531315e-06,4.340304e-06,5.319259e-07,0.002464,5.308757e-06,Benign,0
3012536,6.281540e-07,6.806339e-09,0.000125,3.113900e-06,0.000000,0.000000e+00,1.724675e-08,1.261226e-06,4.298023e-08,3.113900e-06,...,0.084209,0.000000,4.194942e-07,4.194942e-07,6.130762e-07,6.130762e-07,0.000000,4.026154e-07,Benign,0
345028,1.170458e-08,1.145385e-08,0.000058,8.123050e-07,0.000110,8.123050e-07,1.162888e-08,1.162888e-08,1.121200e-08,8.123050e-07,...,0.000000,0.000000,1.094310e-07,1.094310e-07,8.880476e-08,1.165973e-08,0.000048,1.050279e-07,Benign,0
19662289,1.473345e-05,1.410984e-07,0.036372,3.651849e-04,0.027900,3.651849e-04,1.362926e-06,1.927213e-06,8.815569e-08,5.112589e-02,...,0.598319,0.598319,9.839298e-06,9.839298e-06,1.437979e-05,1.437979e-05,0.000000,9.443403e-06,Benign,0


In [45]:
raw_benign_train_samples = df_raw_train[df_raw_train.Label == 0].drop(columns=["Label", "Attack"])
raw_normal_train_samples = df_raw_train.drop(columns=["Label", "Attack"])

raw_train_labels = df_raw_train["Label"]
raw_test_labels = df_raw_test["Label"]

raw_test_samples = df_raw_test.drop(columns=["Label", "Attack"])

In [46]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [47]:
from pyod.models.cblof import CBLOF
n_est = [2,3,5,7,9,10]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    try:
        clf_if.fit(benign_train_samples)
    except Exception as e:
        continue
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [02:36<00:00,  4.33s/it]

{'n_estimators': 10, 'con': 0.01}
0.957345207594555
              precision    recall  f1-score   support

           0     0.9834    0.9973    0.9903    664432
           1     0.9779    0.8764    0.9244     90326

    accuracy                         0.9828    754758
   macro avg     0.9806    0.9369    0.9573    754758
weighted avg     0.9828    0.9828    0.9824    754758



In [48]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    try:
        clf_if.fit(normal_train_samples)
    except Exception as e:
        continue
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [02:55<00:00,  4.86s/it]

{'n_estimators': 9, 'con': 0.01}
0.9559800545767381
              precision    recall  f1-score   support

           0     0.9834    0.9966    0.9900    664432
           1     0.9725    0.8764    0.9220     90326

    accuracy                         0.9822    754758
   macro avg     0.9780    0.9365    0.9560    754758
weighted avg     0.9821    0.9822    0.9818    754758



In [49]:
###  CBLOF RAW

In [50]:
from pyod.models.cblof import CBLOF

n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = CBLOF(n_clusters=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  3%|▎         | 1/36 [00:00<00:08,  4.27it/s]

2


  6%|▌         | 2/36 [00:00<00:08,  4.20it/s]

2


  8%|▊         | 3/36 [00:00<00:08,  4.10it/s]

...

3


 33%|███▎      | 12/36 [00:02<00:05,  4.03it/s]

3


100%|██████████| 36/36 [00:27<00:00,  1.33it/s]

{'n_estimators': 7, 'con': 0.1}
0.8909836489626602
              precision    recall  f1-score   support

           0     0.9750    0.9724    0.9737    332711
           1     0.8006    0.8162    0.8083     45163

    accuracy                         0.9537    377874
   macro avg     0.8878    0.8943    0.8910    377874
weighted avg     0.9541    0.9537    0.9539    377874



In [51]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  3%|▎         | 1/36 [00:00<00:10,  3.26it/s]

2


  6%|▌         | 2/36 [00:00<00:09,  3.55it/s]

2


  8%|▊         | 3/36 [00:00<00:08,  3.74it/s]

...

3


 39%|███▉      | 14/36 [00:04<00:09,  2.23it/s]

5


100%|██████████| 36/36 [00:29<00:00,  1.23it/s]

benign only
{'n_estimators': 10}
0.8911474982860613
              precision    recall  f1-score   support

           0     0.9749    0.9726    0.9737    332711
           1     0.8015    0.8157    0.8085     45163

    accuracy                         0.9538    377874
   macro avg     0.8882    0.8942    0.8911    377874
weighted avg     0.9542    0.9538    0.9540    377874



In [52]:
# HBOS  Embeddings

In [53]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [54]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [06:42<00:00, 11.19s/it]

{'n_estimators': 5, 'con': 0.04}
0.959915863691654
              precision    recall  f1-score   support

           0     0.9846    0.9972    0.9909    664432
           1     0.9772    0.8852    0.9290     90326

    accuracy                         0.9838    754758
   macro avg     0.9809    0.9412    0.9599    754758
weighted avg     0.9837    0.9838    0.9835    754758



In [55]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [07:22<00:00, 12.29s/it]

{'n_estimators': 5, 'con': 0.04}
0.9581931454321198
              precision    recall  f1-score   support

           0     0.9846    0.9963    0.9904    664432
           1     0.9706    0.8853    0.9260     90326

    accuracy                         0.9831    754758
   macro avg     0.9776    0.9408    0.9582    754758
weighted avg     0.9829    0.9831    0.9827    754758



In [56]:
##  HBOS  RAw

In [57]:
from pyod.models.cblof import CBLOF

n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = HBOS(n_bins=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:32<00:00,  1.10it/s]

{'n_estimators': 5, 'con': 0.1}
0.848936269653135
              precision    recall  f1-score   support

           0     0.9581    0.9744    0.9662    332711
           1     0.7843    0.6857    0.7317     45163

    accuracy                         0.9399    377874
   macro avg     0.8712    0.8301    0.8489    377874
weighted avg     0.9373    0.9399    0.9381    377874



In [58]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = HBOS(n_bins=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:33<00:00,  1.08it/s]

benign only
{'n_estimators': 5}
0.8526731297510903
              precision    recall  f1-score   support

           0     0.9621    0.9695    0.9658    332711
           1     0.7618    0.7186    0.7396     45163

    accuracy                         0.9395    377874
   macro avg     0.8620    0.8440    0.8527    377874
weighted avg     0.9382    0.9395    0.9387    377874



In [59]:
##  PCA  Emb

In [60]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [02:40<00:00,  4.45s/it]

{'n_estimators': 10, 'con': 0.05}
0.9602289147639451
              precision    recall  f1-score   support

           0     0.9846    0.9974    0.9909    664432
           1     0.9785    0.8852    0.9295     90326

    accuracy                         0.9839    754758
   macro avg     0.9815    0.9413    0.9602    754758
weighted avg     0.9839    0.9839    0.9836    754758



In [61]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [02:52<00:00,  4.80s/it]

{'n_estimators': 10, 'con': 0.05}
0.9598689229443746
              precision    recall  f1-score   support

           0     0.9846    0.9972    0.9908    664432
           1     0.9771    0.8852    0.9289     90326

    accuracy                         0.9838    754758
   macro avg     0.9808    0.9412    0.9599    754758
weighted avg     0.9837    0.9838    0.9834    754758



In [62]:
##  PCA  RAw

In [63]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples)
   
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:25<00:00,  1.42it/s]

{'n_estimators': 10, 'con': 0.2}
0.8542927068923984
              precision    recall  f1-score   support

           0     0.9646    0.9661    0.9654    332711
           1     0.7476    0.7389    0.7432     45163

    accuracy                         0.9390    377874
   macro avg     0.8561    0.8525    0.8543    377874
weighted avg     0.9387    0.9390    0.9388    377874



In [64]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples)

    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:26<00:00,  1.36it/s]

benign only
{'n_estimators': 20}
0.8543292216769649
              precision    recall  f1-score   support

           0     0.9646    0.9662    0.9654    332711
           1     0.7477    0.7389    0.7433     45163

    accuracy                         0.9390    377874
   macro avg     0.8562    0.8525    0.8543    377874
weighted avg     0.9387    0.9390    0.9388    377874



In [65]:
##  IF  Emb

In [66]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [02:14<00:00,  5.62s/it]


{'n_estimators': 150, 'con': 0.01}
0.9568005465300309
              precision    recall  f1-score   support

           0     0.9826    0.9981    0.9903    664432
           1     0.9840    0.8698    0.9233     90326

    accuracy                         0.9827    754758
   macro avg     0.9833    0.9339    0.9568    754758
weighted avg     0.9827    0.9827    0.9823    754758



In [67]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [02:22<00:00,  5.92s/it]


{'n_estimators': 150, 'con': 0.04}
0.9532847265250207
              precision    recall  f1-score   support

           0     0.9845    0.9939    0.9892    664432
           1     0.9519    0.8852    0.9174     90326

    accuracy                         0.9809    754758
   macro avg     0.9682    0.9396    0.9533    754758
weighted avg     0.9806    0.9809    0.9806    754758



In [68]:
##  IF  Raw

In [69]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples.to_numpy())
   
    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 24/24 [00:35<00:00,  1.48s/it]

{'n_estimators': 100, 'con': 0.2}
0.8384067235263534
              precision    recall  f1-score   support

           0     0.9527    0.9779    0.9651    332711
           1     0.7980    0.6422    0.7117     45163

    accuracy                         0.9378    377874
   macro avg     0.8753    0.8101    0.8384    377874
weighted avg     0.9342    0.9378    0.9348    377874



In [70]:
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples.to_numpy())

    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:37<00:00,  1.57s/it]

benign only
{'n_estimators': 20}
0.849720863814232
              precision    recall  f1-score   support

           0     0.9607    0.9700    0.9654    332711
           1     0.7623    0.7079    0.7341     45163

    accuracy                         0.9387    377874
   macro avg     0.8615    0.8390    0.8497    377874
weighted avg     0.9370    0.9387    0.9377    377874

